In [2]:
import json
import configparser
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments

# 設定
config = configparser.ConfigParser()
print(config)
# config.read('/mnt/d/Python/config/config_V20.txt') # パスの指定が必要です
config.read(r"../config/config_V20.txt")
print(config.sections())
accountID = config['oanda']['account_id']
api_key = config['oanda']['api_key']

api = API(access_token=api_key, environment="practice", headers={"Accept-Datetime-Format":"Unix"})
params = {
    'count': 3,
    'granularity': 'H1',
}
r = instruments.InstrumentsCandles(instrument='USD_JPY', params=params)
response = api.request(r)
print(json.dumps(response, indent=2))

['oanda']
{
  "instrument": "USD_JPY",
  "granularity": "H1",
  "candles": [
    {
      "complete": true,
      "volume": 3494,
      "time": "1580475600.000000000",
      "mid": {
        "o": "108.964",
        "h": "108.968",
        "l": "108.826",
        "c": "108.852"
      }
    },
    {
      "complete": true,
      "volume": 5131,
      "time": "1580479200.000000000",
      "mid": {
        "o": "108.853",
        "h": "108.888",
        "l": "108.722",
        "c": "108.758"
      }
    },
    {
      "complete": false,
      "volume": 4654,
      "time": "1580482800.000000000",
      "mid": {
        "o": "108.756",
        "h": "108.774",
        "l": "108.667",
        "c": "108.679"
      }
    }
  ]
}


In [3]:
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import pandas as pd
from datetime import datetime
# start～endまでのデータ取得
def get_period_data(start, end, minute, instrument='USD_JPY'):
    timestamp = start.timestamp()
    concats = []
    count = 5000
    while True:
        df, last_timestamp = send_api(count, timestamp, minute, instrument)
        concats.append(df)
        print(df)
        if last_timestamp > end.timestamp() or len(df) < count:
            break
        timestamp = last_timestamp + (60 * minute)
    df = pd.concat(concats)
    if end is None:
        print(df)
        return df
    else:
        return df[df.index < end]
# 時間足のdfを取得
def send_api(count, start, minute, instrument):
    # oandaへのリクエストの送信
    #api_key = '取得したアクセストークンを入れてください'
    api = API(access_token=api_key, environment="practice", headers={"Accept-Datetime-Format":"Unix"})
    if minute == 1/60:
        granularity = 'S5'
    elif minute == 1:
        granularity = 'M1'
    elif minute == 5:
        granularity = 'M5'
    elif minute == 15:
        granularity = 'M15'
    elif minute == 60:
        granularity = 'H1'
    params = {
        'count': count,
        'granularity': granularity,
    }
    if start is not None:
        params['from'] = start
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    response = api.request(r)

    # レスポンスの整形
    def join_json(candle):
        tmp = candle['mid']
        tmp['time'] = candle['time']
        tmp['v'] = candle['volume']
        tmp['complete'] = candle['complete']
        return tmp
    data_list = [join_json(candle) for candle in response['candles']]
    df = pd.DataFrame(data_list)
    last_timestamp = int(float(df.iloc[-1]['time']))
    
    # 型変更
    df['time'] = df['time'].astype('float64')
    df['o'] = df['o'].astype('float64')
    df['h'] = df['h'].astype('float64')
    df['l'] = df['l'].astype('float64')
    df['c'] = df['c'].astype('float64')
    df['v'] = df['v'].astype('float64')

    # タイムゾーンの変更、インデックス化
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['time'] = df['time'] + pd.Timedelta('09:00:00') # 日本時間へ変換
    df.set_index('time', inplace=True)                 # 時間をインデックスにする
    df = df.loc[:,['o','h','l', 'c', 'v', 'complete']] # 列の順番変更
    df = df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'amount'})

    return df, last_timestamp
minute = 60
start = datetime.strptime('2020-01-02 20:00:00', '%Y-%m-%d %H:%M:%S')
end = datetime.now()
#end = datetime.strptime('2019-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
df = get_period_data(start, end, minute, instrument='USD_JPY')
df

                        open     high      low    close  amount  complete
time                                                                     
2020-01-02 20:00:00  108.804  108.812  108.772  108.772   225.0      True
2020-01-02 21:00:00  108.774  108.807  108.756  108.788   213.0      True
2020-01-02 22:00:00  108.786  108.790  108.658  108.694   395.0      True
2020-01-02 23:00:00  108.697  108.748  108.637  108.675   646.0      True
2020-01-03 00:00:00  108.672  108.681  108.212  108.274  1442.0      True
...                      ...      ...      ...      ...     ...       ...
2020-01-31 20:00:00  108.946  108.982  108.916  108.917  2638.0      True
2020-01-31 21:00:00  108.918  108.965  108.914  108.964  2055.0      True
2020-01-31 22:00:00  108.964  108.968  108.826  108.852  3494.0      True
2020-01-31 23:00:00  108.853  108.888  108.722  108.758  5131.0      True
2020-02-01 00:00:00  108.756  108.774  108.667  108.676  4664.0     False

[509 rows x 6 columns]


,open,high,low,close,amount,complete
time,,,,,,
2020-01-02 20:00:00,108.804,108.812,108.772,108.772,225.0,True
2020-01-02 21:00:00,108.774,108.807,108.756,108.788,213.0,True
2020-01-02 22:00:00,108.786,108.790,108.658,108.694,395.0,True
2020-01-02 23:00:00,108.697,108.748,108.637,108.675,646.0,True
2020-01-03 00:00:00,108.672,108.681,108.212,108.274,1442.0,True
...,...,...,...,...,...,...
2020-01-31 20:00:00,108.946,108.982,108.916,108.917,2638.0,True
2020-01-31 21:00:00,108.918,108.965,108.914,108.964,2055.0,True
2020-01-31 22:00:00,108.964,108.968,108.826,108.852,3494.0,True


In [4]:
df = df.reset_index().drop_duplicates(subset='time',keep='first').set_index('time')
df

,open,high,low,close,amount,complete
time,,,,,,
2020-01-02 20:00:00,108.804,108.812,108.772,108.772,225.0,True
2020-01-02 21:00:00,108.774,108.807,108.756,108.788,213.0,True
2020-01-02 22:00:00,108.786,108.790,108.658,108.694,395.0,True
2020-01-02 23:00:00,108.697,108.748,108.637,108.675,646.0,True
2020-01-03 00:00:00,108.672,108.681,108.212,108.274,1442.0,True
...,...,...,...,...,...,...
2020-01-31 20:00:00,108.946,108.982,108.916,108.917,2638.0,True
2020-01-31 21:00:00,108.918,108.965,108.914,108.964,2055.0,True
2020-01-31 22:00:00,108.964,108.968,108.826,108.852,3494.0,True


In [5]:
df.to_csv('20200102_200000_20200201_120000_H1.csv')